In [29]:
import utils
import pandas as pd
import pickle

In [30]:
def load_corpus_xlsx(filename):
    corpus = pd.read_excel(filename, dtype={'msg': str})
    list_label = corpus['class'].to_list()
    list_content = corpus['mgs'].to_list()
    return list_label, list_content
classifier_list = ['nb']

In [31]:
list_label, list_content = load_corpus_xlsx('../message_modified_v1.2.xlsx')

In [32]:
list_content = utils.entity_tagging(list_content)
list_content

['qc lo lắng khi hết data mobifone tặng bạn 1gb data tốc độ cao và number phút gọi nội mạng miễn phí mỗi ngày qua tổng đài number soạn kmb gửi number đăng ký dv mobiradio miễn phí ngày đầu duy trì dv date tiếp theo để nhận data sau km currency ngày lh number từ chối qc soạn tc voice gửi number',
 'cát khá thơm nhà mình nuôi number bé mà k bị thúi 🤣 hạt to nên cũng dế dọn dẹp recomment mn nên mua nha',
 'cũng tạm ổn hahaaa món nào mua của shop thấy cubgx oke hết á có điều mũ khá là nhăn nheo í thấy không bằng cái mũ 20k ngoài chợ luôn chất vải mũ siêu mỏng còn nhăn nheo cực kì mất form',
 'nước tẩy trang quốc dân giá rẻ chất lượng tốt dùng tẩy trang hằng ngày thì number bông là oklah🌟🌟🌟',
 'cảm ơn shop va shopee nhiều nha mịnh da nhan duoc san pham kiểm tra ok shop mall mà gởi hamg hoi lâu',
 'giao hàng nhanh giá hợp lý hạt cũng khá ổn trong tầm giá nhưng nên đổi qua lại các loại hạt mỗi loại có ưu nhược điểm riêng không nên cho mèo ăn một loại quá lâu khi đổi nên trộn number loại với n

In [33]:
def train_evaluation(list_label, list_content, vectorize_method, classifier_list):

    if vectorize_method == 'bow':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'bow')
    for i in classifier_list:
        list_false_positive, list_false_negative, list_true_positive, list_true_negative = \
        utils.kfold_classification(list_content, list_content_vec, list_label, i, 5, vectorize_method)
        print('Evaluation for ' + i.upper())
        utils.evaluation(list_false_positive, list_false_negative, list_true_positive, list_true_negative)
        print('-------------------------------------------------')

In [34]:
def train_entire_dataset(list_content, list_label, vectorize_method):
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, vectorize_method)
    clf = utils.build_classifier_nb(list_content_vec, list_label)
    pickle.dump(clf, open(r'C:\Users\MINH.NV193012\Desktop\20222\ProjectAI_Application\bow_svm_clf.pkl', 'wb'))
    print('Traning Done!')

In [35]:
train_entire_dataset(list_content, list_label,'bow')

Traning Done!


In [36]:
train_evaluation(list_label, list_content, 'bow', ['nb'])

Evaluation for NB
False Positive Rate: 2.8962959125254564%
False Negative Rate: 66.42654656916574%
True Positive Rate: 33.57345343083425%
True Negative Rate: 97.10370408747454%
-------------------------------------------------
